In [1]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

Using TensorFlow backend.


In [2]:
lines= pd.read_table('uniquew.txt', names=['eng', 'tel'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


In [3]:
lines.shape

(2232, 2)

In [4]:
lines.eng=lines.eng.str.lower()#apply(lambda x: x.lower())
lines.tel=lines.tel.str.lower()

In [5]:
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', str(x)))
lines.tel=lines.tel.apply(lambda x: re.sub("'", '', str(x)))

In [60]:
exclude = set(string.punctuation)
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.tel=lines.tel.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [61]:
lines.eng

0                                             definitely
1                                             he hung up
2                                             i ran home
3                                             who are we
4                                            are you mad
5                                          he touched me
6                                          my head hurts
7                                         i drank coffee
8                                        how tall is she
9                                         theyre animals
10                                      can you see that
11                                      i began to speak
12                                      i dislike coffee
13                                       im hungry again
14                                      i dont accept it
15                                      whats in the box
16                                     which is your pen
17                             

In [62]:
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
lines.tel = lines.tel.apply(lambda x: re.sub("[0123456789]", "", x))

In [63]:
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.tel=lines.tel.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.tel=lines.tel.apply(lambda x: re.sub(" +", " ", x))

lines.tel = lines.tel.apply(lambda x : 'START_ '+ x + ' _END')

In [64]:
lines.sample(10)

,eng,tel
68,where did you put my book,START_ నా పుస్తకం ఎక్కడ పెట్టావ్ _END
75,we dont understand french,START_ మాకు ఫ్రెంచి అర్ధం కాదు _END
63,we havent slept in days,START_ మేము రోజుల తరబడి నిద్రపోలేదు _END
126,it was apparent that there was no way out,START_ వేరే దారి లేదని స్పష్టంగా తెలుస్తుంది _END
80,you must stay where you are,START_ నువ్వు ఎక్కడ ఉన్నవో అక్కడే ఉండు _END
45,i dont trust that guy,START_ నేను వాడిని నమ్మను _END
56,she scared the cat away,START_ ఆవిడ పిల్లిని భయపెట్టి తరిమేసింది _END
13,im hungry again,START_ నాకు మళ్ళా ఆకలి వేస్తుంది _END
81,dont interfere with my work,START_ నా పని లో అడ్డు రాకు _END
57,the work is almost done,START_ పని దాదాపుగా అయిపోయింది _END


In [46]:
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

In [47]:
all_Telugu_words=set()
for tel in lines.tel:
    for word in tel.split():
        if word not in all_Telugu_words:
            all_Telugu_words.add(word)

In [48]:
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
print(max_length_src)

13


In [49]:
lenght_list=[]
for l in lines.tel:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)

print(max_length_tar)

11


In [50]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_Telugu_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_Telugu_words)
print(num_encoder_tokens, num_decoder_tokens)
num_decoder_tokens += 1

348 403


In [51]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [52]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [53]:
lines = shuffle(lines)
print(lines.head(10))

                                         eng  \
112       i dont have time to talk right now   
124  which color do you prefer blue or green   
93            you may go swimming or fishing   
91            we played basketball yesterday   
75                 we dont understand french   
16                         which is your pen   
67                 when did you quit smoking   
32                       ill be very careful   
81               dont interfere with my work   
99           i cant keep you here any longer   

                                                   tel  
112      START_ నాకు ఇప్పుడు మాట్లాడేంత సమయం లేదు _END  
124  START_ నీకు ఏ రంగు అంటే ఇష్టం నీలమా లేక పచ్ఛా ...  
93   START_ నువ్వు ఈత కొట్టడానికో లేక చేపలు పట్టడాన...  
91          START_ మేము నిన్న బాస్కెట్ బాల్ ఆడాము _END  
75                 START_ మాకు ఫ్రెంచి అర్ధం కాదు _END  
16                              START_ నీ కలం ఏది _END  
67               START_ పొగ తాగడం ఎప్పుడు ఆపేశావ్ _END  
32             

In [54]:
X, y = lines.eng, lines.tel
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
print(X_train.shape, X_test.shape)

(124,) (14,)


In [22]:
X_train.to_pickle('Weights_Tel/X_train.pkl')
X_test.to_pickle('Weights_Tel/X_test.pkl')

In [23]:
def generate_batch(X = X_train, y = y_train, batch_size = 10):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src+1),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar+1),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar+1, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                print( input_text)
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [24]:
latent_dim = 50

In [25]:

# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens+1, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

Instructions for updating:
Colocations handled automatically by placer.


In [26]:
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [27]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [28]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 10
epochs = 50

train_samples,val_samples

(124, 14)

In [29]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = int(train_samples/batch_size),
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = int(val_samples/batch_size))


Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/50
12/12 [==============================] - 5s 394ms/step - loss: 5.9867 - acc: 0.1727 - val_loss: 5.9441 - val_acc: 0.2083
Epoch 2/50
12/12 [==============================] - 1s 60ms/step - loss: 5.6793 - acc: 0.1933 - val_loss: 5.6240 - val_acc: 0.1538
Epoch 3/50
12/12 [==============================] - 1s 51ms/step - loss: 5.1944 - acc: 0.1936 - val_loss: 5.3746 - val_acc: 0.2083
Epoch 4/50
12/12 [==============================] - 1s 63ms/step - loss: 5.0787 - acc: 0.1959 - val_loss: 5.9172 - val_acc: 0.1538
Epoch 5/50
12/12 [==============================] - 1s 64ms/step - loss: 5.0315 - acc: 0.1959 - val_loss: 5.5098 - val_acc: 0.2083
Epoch 6/50
12/12 [==============================] - 1s 54ms/step - loss: 5.0104 - acc: 0.1945 - val_loss: 6.1832 - val_acc: 0.1538
Epoch 7/50
12/12 [==============================] - 1s 56ms/step - loss: 4.9687 - acc: 

In [30]:
model.save_weights('nmt_weights.h5')

In [31]:
model.load_weights('nmt_weights.h5')

In [32]:
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [33]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [34]:
train_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [35]:
count=0
while(k<182):
    k+=1
    (input_seq, actual_output), _ = next(train_gen)
    decoded_sentence = decode_sequence(input_seq)
    inp=X_test[k:k+1].values[0]
    act=y_test[k:k+1].values[0][6:-4]
    pred=decoded_sentence[:-4]
    print('Input English sentence:', inp)
    print('Actual Tel Translation:',act )
    print('Predicted Tel Translation:', pred)
    
    if(act==pred):
        count+=1
count

Input English sentence: i began to speak
Actual Tel Translation:  నేను మాట్లాడటం మొదలుపెట్టాను 
Predicted Tel Translation:  ఆమె నిన్న లో 
Input English sentence: help came too late
Actual Tel Translation:  సహాయం చాలా ఆలస్యముగా వచ్చింది 
Predicted Tel Translation:  నీ 
Input English sentence: he made her a bookshelf
Actual Tel Translation:  అతను ఆమెకి పుస్తకాల అర తయారు చేసాడు 
Predicted Tel Translation:  నాకు నాకు కావాలని కాదు 
Input English sentence: he touched me
Actual Tel Translation:  అతను నన్ను తాకాడు 
Predicted Tel Translation:  నాకు 
Input English sentence: are you mad
Actual Tel Translation:  కోపమొచ్చిందా 
Predicted Tel Translation:  నువ్వు చాలా లేదు 
Input English sentence: when youre a father youll understand
Actual Tel Translation:  నువ్వు నాన్న అయినప్పుడు నీకు అర్ధం అవుతుంది 
Predicted Tel Translation:  ఆమె నిన్న లో 
Input English sentence: it actually isnt that simple
Actual Tel Translation:  అది అంత సులభం ఏం కాదు 
Predicted Tel Translation:  నువ్వు చాలా కాదు 
Input Englis

IndexError: index 0 is out of bounds for axis 0 with size 0